# Kepler Framework, Emulate v8

In [14]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [16]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = '../../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

In [17]:
tpath_data17 = 'output/emulation/data17_table_v8.h5'
tpath_data18 = 'output/emulation/data18_table_v8.h5'
tpath_mc16   = 'output/emulation/mc16_table_v8.h5'

## Setup Chains:

In [18]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v8_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v8_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v8_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v8_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v8" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v8_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v8_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v8_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v8_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v8_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [19]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [20]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:27<00:00, 21.78s/it]


In [21]:
data17_df.shape

(43311283, 111)

### Validate:

In [22]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v8.root')

2022-03-13 10:04:52,236 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v8.root


In [23]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v8_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v8'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:57<00:00, 23.57s/it]


HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.5668  | 29985882/32047563 |
|   L2   | 93.3285  | 29909501/32047563 |
| EFCalo | 92.0150  | 29488558/32047563 |
|  HLT   | 90.0519  | 28859429/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:24<00:00, 16.96s/it]


HLT_e28_lhtight_nod0_ringer_v8_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2219  | 21450188/22765602 |
|   L2   | 94.1095  | 21424585/22765602 |
| EFCalo | 93.1853  | 21214197/22765602 |
|  HLT   | 87.0128  | 19808994/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:06<00:00,  1.25s/it]


HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.3617  | 1331700/1367786 |
|   L2   | 97.2733  | 1330490/1367786 |
| EFCalo | 95.1854  | 1301933/1367786 |
|  HLT   | 91.9068  | 1257088/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 19.00it/s]

HLT_e140_lhloose_nod0_ringer_v8
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 98.0162  | 59340/60541  |
|   L2   | 97.9501  | 59300/60541  |
| EFCalo | 96.6287  | 58500/60541  |
|  HLT   | 94.6598  | 57308/60541  |
+--------+----------+--------------+


In [24]:
eff_data17_Zee.save()

In [25]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v8.root')

2022-03-13 10:09:14,586 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v8.root


In [26]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI', pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v8_ivarloose' , pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI', pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v8'          , pidname='!el_lhvloose', reset=True)

Filling...: 100%|██████████| 5/5 [00:22<00:00,  4.54s/it]


HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  2.2395  | 231784/10349992  |
|   L2   |  2.1632  | 223892/10349992  |
| EFCalo |  1.9647  | 203343/10349992  |
|  HLT   |  0.1684  |  17430/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:20<00:00,  4.08s/it]


HLT_e28_lhtight_nod0_ringer_v8_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.8360  |  77362/9254300  |
|   L2   |  0.7923  |  73320/9254300  |
| EFCalo |  0.6545  |  60568/9254300  |
|  HLT   |  0.0018  |   167/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.44s/it]


HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.8335  |  65446/7852181  |
|   L2   |  0.7826  |  61451/7852181  |
| EFCalo |  0.6158  |  48353/7852181  |
|  HLT   |  0.0026  |   203/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.58s/it]

HLT_e140_lhloose_nod0_ringer_v8
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  2.0564  |  115858/5634041 |
|   L2   |  1.9617  |  110523/5634041 |
| EFCalo |  1.6871  |  95050/5634041  |
|  HLT   |  0.0488  |   2752/5634041  |
+--------+----------+-----------------+


In [27]:
eff_data17_JF17.save()

In [28]:
del data17_df

## Emulate 2018:

In [29]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [30]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:27<00:00, 21.84s/it]


In [31]:
data18_df.shape

(41873973, 111)

In [32]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v8.root')

2022-03-13 10:14:51,267 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v8.root


In [33]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v8_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v8'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:59<00:00, 23.96s/it]


HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.6121  | 30536720/32620495 |
|   L2   | 93.3829  | 30461951/32620495 |
| EFCalo | 92.0468  | 30026120/32620495 |
|  HLT   | 90.5900  | 29550912/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:25<00:00, 17.14s/it]


HLT_e28_lhtight_nod0_ringer_v8_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.1907  | 21959450/23313826 |
|   L2   | 94.0962  | 21937429/23313826 |
| EFCalo | 93.2973  | 21751180/23313826 |
|  HLT   | 87.5868  | 20419832/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:06<00:00,  1.24s/it]


HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.3481  | 1345035/1381675 |
|   L2   | 97.2618  | 1343842/1381675 |
| EFCalo | 95.1764  | 1315029/1381675 |
|  HLT   | 92.1905  | 1273773/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 19.05it/s]

HLT_e140_lhloose_nod0_ringer_v8
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 98.0990  | 59241/60389  |
|   L2   | 98.0046  | 59184/60389  |
| EFCalo | 96.8223  | 58470/60389  |
|  HLT   | 94.9743  | 57354/60389  |
+--------+----------+--------------+


In [34]:
eff_data18_Zee.save()

In [35]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v8.root')

2022-03-13 10:19:19,276 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v8.root


In [36]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI', pidname='!el_lhvloose', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v8_ivarloose' , pidname='!el_lhvloose', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI', pidname='!el_lhvloose', reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v8'          , pidname='!el_lhvloose', reset=True)

Filling...: 100%|██████████| 5/5 [00:18<00:00,  3.71s/it]


HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  2.7125  |  229666/8467098 |
|   L2   |  2.6304  |  222722/8467098 |
| EFCalo |  2.4375  |  206382/8467098 |
|  HLT   |  0.6946  |  58815/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.32s/it]


HLT_e28_lhtight_nod0_ringer_v8_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  1.1949  |  91156/7628804  |
|   L2   |  1.1537  |  88015/7628804  |
| EFCalo |  1.0213  |  77911/7628804  |
|  HLT   |  0.0464  |   3537/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:14<00:00,  2.85s/it]


HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  1.1867  |  77868/6561985  |
|   L2   |  1.1387  |  74723/6561985  |
| EFCalo |  0.9812  |  64384/6561985  |
|  HLT   |  0.0574  |   3766/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.15s/it]

HLT_e140_lhloose_nod0_ringer_v8
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  2.6206  |  125175/4776518 |
|   L2   |  2.5184  |  120294/4776518 |
| EFCalo |  2.2579  |  107848/4776518 |
|  HLT   |  0.3619  |  17287/4776518  |
+--------+----------+-----------------+


In [37]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [38]:
mc16_df = pd.concat( (load_hdf(dpath_mc16), load_hdf(tpath_mc16)), axis=1)

In [39]:
emulate(mc16_df)

Emulate...: 100%|██████████| 4/4 [00:00<00:00, 38.03it/s]


In [40]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_v8.root')

2022-03-13 10:20:36,020 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/mc16_boosted_efficiency_v8.root


In [41]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v8_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v8'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [00:00<00:00, 16.15it/s]


HLT_e17_lhvloose_nod0_ringer_v8_L1EM15VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.7303  | 78407/78619  |
| L2Calo | 96.3686  | 75764/78619  |
|   L2   | 96.3584  | 75756/78619  |
| EFCalo | 75.5263  | 59378/78619  |
|  HLT   | 74.4756  | 58552/78619  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 18.64it/s]


HLT_e28_lhtight_nod0_ringer_v8_ivarloose
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.8427  | 72990/73105  |
| L2Calo | 74.5435  | 54495/73105  |
|   L2   | 74.5380  | 54491/73105  |
| EFCalo | 60.6320  | 44325/73105  |
|  HLT   | 58.1232  | 42491/73105  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 18.21it/s]


HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9635  | 73911/73938  |
| L2Calo | 77.2526  | 57119/73938  |
|   L2   | 77.2472  | 57115/73938  |
| EFCalo | 61.9100  | 45775/73938  |
|  HLT   | 60.2978  | 44583/73938  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 18.32it/s]

HLT_e140_lhloose_nod0_ringer_v8
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9882  | 67947/67955  |
| L2Calo | 94.6553  | 64323/67955  |
|   L2   | 94.6509  | 64320/67955  |
| EFCalo | 73.5060  | 49951/67955  |
|  HLT   | 72.0918  | 48990/67955  |
+--------+----------+--------------+


In [42]:
eff_mc16.save()